# Project Trinity

#### University of Pavia

- Michele Ventimiglia
- Chiara Curgu

## Setup

Install required libraries

In [1]:
#!python -m pip install --upgrade --quiet numpy scipy matplotlib ipykernel ipywidgets

Libraries import

In [2]:
import sys
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from typing import Tuple, Dict, List
from ipywidgets import interact, FloatSlider, Button

Versions info

In [3]:
print(f"[i] Python v{sys.version}")
print(f"[i] NumPy v{np.__version__}")
print(f"[i] SciPy v{scipy.__version__}")

[i] Python v3.11.8 (tags/v3.11.8:db85d51, Feb  6 2024, 22:03:32) [MSC v.1937 64 bit (AMD64)]
[i] NumPy v1.26.4
[i] SciPy v1.12.0


## Model

Morris-Lecar model equations

In [4]:
def morris_lecar(
        V_w: Tuple[float, float],
        t: float,
        params: Dict[str, float]
    ) -> Tuple[float, float]:
    """
    Implements the Morris-Lecar model to calculate the time evolution of membrane potential (V) 
    and gating variable (w) in a neuron based on differential equations.
    
    The Morris-Lecar model is a simplified model of neuronal activity that describes how the membrane potential evolves over time in response to external currents and the neuron's own ion channels.
    
    Args:
        V_w (Tuple[float, float]): A tuple containing the current values of the membrane potential (V) and the gating variable (w).
        t (float): The current time. Not used in the calculation directly but required for compatibility with ODE solvers.
        params (Dict[str, float]): A dictionary containing parameters of the model. Expected keys are:
            - 'I_ext': External current applied to the neuron.
            - 'V1', 'V2', 'V3', 'V4': Parameters for the voltage-dependent activation/inactivation variables.
            - 'g_Ca', 'g_K', 'g_L': Conductances for Ca2+, K+, and leak channels, respectively.
            - 'V_Ca', 'V_K', 'V_L': Reversal potentials for Ca2+, K+, and leak channels, respectively.
            - 'C': Membrane capacitance.
            - 'phi': Rate constant for the gating variable w.
    
    Returns:
        Tuple[float, float]: The derivatives of the membrane potential (dV/dt) and the gating variable (dw/dt) with respect to time.
    """
    
    V, w = V_w # Unpack the membrane potential and gating variable
    I_ext = params['I_ext']
    
    # Calculate the steady-state activation and time constant for the gating variable
    m_inf = 0.5 * (1.0 + np.tanh((V - params['V1']) / params['V2']))
    w_inf = 0.5 * (1.0 + np.tanh((V - params['V3']) / params['V4']))
    tau_w = 1 / (params['phi'] * np.cosh((V - params['V3']) / (2 * params['V4'])))
    
    # Calculate the derivatives of V and w using the Morris-Lecar equations
    dV_dt = (I_ext - params['g_Ca'] * m_inf * (V - params['V_Ca']) - params['g_K'] * w * (V - params['V_K']) - params['g_L'] * (V - params['V_L'])) / params['C']
    dw_dt = params['phi'] * (w_inf - w) / tau_w
    
    return [dV_dt, dw_dt]

### Parameters

In [5]:
def default_parameters() -> dict:
    """
    Default parameters and initial conditions for the Morris-Lecar model.
    
    Returns:
        dict: A dictionary of default parameters.
    """
    
    return {
        "C": 20,  # Membrane capacitance, uF/cm^2
        "V_Ca": 120,  # Reversal potential for calcium, mV
        "V_K": -84,  # Reversal potential for potassium, mV
        "V_L": -60,  # Reversal potential for leak channels, mV
        "g_Ca": 4.4,  # Maximum conductance for calcium, mS/cm^2
        "g_K": 8,  # Maximum conductance for potassium, mS/cm^2
        "g_L": 2,  # Maximum conductance for leak, mS/cm^2
        "phi": 0.04,  # Temperature coefficient
        "V1": -1.2,  # Parameter for activation/inactivation
        "V2": 18,  # Parameter for activation/inactivation
        "V3": 2,  # Parameter for activation/inactivation
        "V4": 30,  # Parameter for activation/inactivation
        "I_ext": 0.1,  # External current, uA/cm^2
        "V0": -70*(10**-3),  # Initial membrane potential, V
        "w0": 0.0  # Initial fraction of open potassium channels
    }

## Processing

Solve ODE

In [6]:
def solve_model(I_ext: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Solves the Morris-Lecar model ordinary differential equations (ODEs) for given parameter values.
    
    This function integrates the Morris-Lecar model over a specified time period using the given parameters for the model. It customizes the model by allowing the user to specify values for the parameters V1, V2, V3, and V4, which affect the dynamics of the membrane potential and recovery variable.
    
    Args:
        I_ext (float): External current.
    
    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray]: A tuple containing three np.ndarrays. The first array is the time vector. The second and third arrays are the time series of the membrane potential (V) and recovery variable (w), respectively.
    """
    
    # Define the time vector over which to solve the ODEs
    t = np.linspace(0, 5000, 10000)
    
    # Retrieve default parameters and update them with the given values
    params = default_parameters()
    params.update({'I_ext': I_ext})
    
    # Define the initial conditions for the integration
    initial_conditions = [params['V0'], params['w0']]
    
    # Solve the ODEs using scipy's odeint function
    sol = odeint(morris_lecar, initial_conditions, t, args=(params,))
    
    # Return the time vector, membrane potential, and recovery variable
    return t, sol[:, 0], sol[:, 1]

Data Analysis

In [7]:
def analyze_data(
        V: np.ndarray,
        w: np.ndarray,
        t: np.ndarray
    ) -> List[Dict[str, float]]:
    """
    Analyzes the data from the Morris-Lecar model simulation. Now, it also finds the critical points
    where the voltage and recovery variable curves intercept each other.
    
    Args:
        V (np.ndarray): Array of membrane potentials over time.
        w (np.ndarray): Array of recovery variable values over time.
        t (np.ndarray): Array representing the simulation time points.
    
    Returns:
        List[Dict[str, float]]: The critical points.
    """
    # Calculate differences between V and w to find where they cross
    differences = V - w
    
    # Identifying the sign changes in differences, which indicate crossing points
    sign_changes = np.diff(np.sign(differences)) != 0

    # Getting the indices right before the crossing points
    cross_indices = np.where(sign_changes)[0]
    
    # Collecting critical points (V, w, t) at crossing points
    critical_points = [{'V': V[i], 'w': w[i], 't': t[i]} for i in cross_indices]

    return critical_points

## Results

In [8]:
global current_plot_state

Function to save the plot

In [9]:
def save_plot(filename: str = 'plot') -> None:
    
    global current_plot_state
    
    if current_plot_state is None:
        print("No plot to save.")
        return
    
    I_ext, V, w, t = current_plot_state
    
    filename = f"{filename}_I_ext_{I_ext}.png"
    
    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.plot(t, V)
    plt.title('Membrane Potential Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.grid()

    plt.subplot(1, 3, 2)
    plt.plot(V, w)
    plt.title('Phase Plane')
    plt.xlabel('Membrane Potential (mV)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    plt.subplot(1, 3, 3)
    plt.plot(t, w)
    plt.title('Recovery Variable Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    plt.savefig(f"docs/img/{filename}")
    plt.close()
    
    print(f"Plot saved as {filename}")

Plotting

In [10]:
def interactive_plot(I_ext: float) -> None:
    
    global current_plot_state
    
    t, V, w = solve_model(I_ext)
    critical_points = analyze_data(V, w, t)
    
    current_plot_state = I_ext, V, w, t
    
    # Plotting
    plt.figure(figsize=(18, 6))

    # Plot V(t)
    plt.subplot(1, 3, 1)
    plt.plot(t, V)
    plt.title('Membrane Potential Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.grid()

    # Plot phase plane trajectory
    plt.subplot(1, 3, 2)
    plt.plot(V, w)
    plt.title('Phase Plane')
    plt.xlabel('Membrane Potential (mV)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    # Plot w(t)
    plt.subplot(1, 3, 3)
    plt.plot(t, w)
    plt.title('Recovery Variable Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()
    
    plt.show()

    # Print analysis results
    for i, point in enumerate(critical_points):
        if i != 0: # skip initial graphs value
            print(f"Critical point {i}: {point}")


In [11]:
params = default_parameters()

In [12]:
interact(
    interactive_plot,
    I_ext = FloatSlider(
        value = params['I_ext'],
        min = 0,
        max = 100,
        step = 10))

save_button = Button(description="Save Plot")
save_button.on_click(lambda b: save_plot())
display(save_button)

interactive(children=(FloatSlider(value=0.1, description='I_ext', step=10.0), Output()), _dom_classes=('widget…

Button(description='Save Plot', style=ButtonStyle())